# DataFrame Manipulations

> Data Cleaning in Pandas requires the manipulation of `DataFrames` through adding, removing, and altering data. Techniques covered in this lesson include adding new columns and rows, dropping unnecessary ones, and applying elementwise modifications to the data, for example by applying a function to each element. Merging data from different sources and grouping for detailed analysis are also key aspects. These processes are fundamental to maintaining data integrity and usability in Pandas.

## Adding and Removing Data


### Adding a Column

#### 1. Direct Assignment:
The simplest way to add a new column is by assigning a list or array to a new column name. 



In [2]:
import pandas as pd
my_dict = {'animal': ['Dog', 'Cat', 'Pig', 'Lion'], 'age': [2, 4, 5, 7]}
df = pd.DataFrame(my_dict)
df

,animal,age
0,Dog,2
1,Cat,4
2,Pig,5
3,Lion,7


In [3]:
df['type'] = ['Domestic', 'Domestic', 'Domestic', 'Wild']
df

,animal,age,type
0,Dog,2,Domestic
1,Cat,4,Domestic
2,Pig,5,Domestic
3,Lion,7,Wild




### Adding Rows



#### Using `pd.concat()`
> Since Pandas v2, the only permitted way to add new rows or append one `DataFrame` to another is the `pd.concat()` function. Older versions of Pandas have a method called `.append()` that allowed you to add a single row, but this has now been removed. 


In [4]:
my_dict_2 = {'animal': ['Elephant'], 'age': [3]}
additional_data = pd.DataFrame(my_dict_2)

In [5]:
additional_data

,animal,age
0,Elephant,3


In [6]:
base_df = pd.concat([df, additional_data], ignore_index=True)
base_df

,animal,age,type
0,Dog,2,Domestic
1,Cat,4,Domestic
2,Pig,5,Domestic
3,Lion,7,Wild
4,Elephant,3,NaN


In [7]:
additional_data_2 = pd.DataFrame([['Horse', 3, 'Sport']], columns=base_df.columns)
base_df = pd.concat([base_df, additional_data_2], ignore_index=True)
base_df

,animal,age,type
0,Dog,2,Domestic
1,Cat,4,Domestic
2,Pig,5,Domestic
3,Lion,7,Wild
4,Elephant,3,NaN
5,Horse,3,Sport



An important consideration here is the `ignore_index` parameter, which is used to control how the index is handled during concatenation.

- `ignore_index=True`: With this setting, Pandas resets the index to the default integer index. This means the resulting `DataFrame` will have a new index ranging from `0` to `n-1`, where `n` is the length of the `DataFrame`. Values in the second `DataFrame` are appended to the end of the first `DataFrame`, and the index is renumbered to reflect this continuous sequence.

- `ignore_index=False` (default behavior): When set to `False`, Pandas preserves the original indices of the concatenated `DataFrames`. In this case, the indices from each `DataFrame` are maintained, and the final `DataFrame` reflects these original indices. This might lead to duplicate index values if the original `DataFrames` have overlapping indices. The data are still combined as expected, with the contents of the second `DataFrame` following those of the first, but the original order and index values from each `DataFrame` are kept intact.

The choice between `ignore_index=True` and `ignore_index=False` depends on whether the original index carries meaningful information for your data and whether unique, non-overlapping indices are needed post-concatenation. 


In [9]:
# Creating two sample DataFrames
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [5, 6], 'B': [7, 8]})

In [10]:
df1

,A,B
0,1,3
1,2,4


In [11]:
df2

,A,B
0,5,7
1,6,8


In [12]:
# Concatenating without ignore_index
result_without_ignore_index = pd.concat([df1, df2])
print("With ignore_index=False:\n", result_without_ignore_index)

# Concatenating with ignore_index
result_with_ignore_index = pd.concat([df1, df2], ignore_index=True)
print("\nWith ignore_index=True:\n", result_with_ignore_index)

With ignore_index=False:
    A  B
0  1  3
1  2  4
0  5  7
1  6  8

With ignore_index=True:
    A  B
0  1  3
1  2  4
2  5  7
3  6  8




### Dropping Rows and Columns

- **Dropping Rows:**
  - Rows can be removed using the `drop()` method, specifying the index labels and `axis=0`
  - Example:
    ```python
    df.drop(index=[0, 1], inplace=True)  # Drops rows with index 0 and 1
    ```

- **Dropping Columns:**
  - To drop columns, use the `drop()` method with the column names and `axis=1`
  - Example:
    ```python
    df.drop(columns=['Column1', 'Column2'], inplace=True)
    ```

- **Using `dropna()`:**
  - The `dropna()` method is useful for removing rows or columns with missing values
  - Example:
    ```python
    df.dropna(axis=0, inplace=True)  # Drops rows with any NaN values
    df.dropna(axis=1, inplace=True)  # Drops columns with any NaN values
    ```

These tools are integral to shaping your dataset into the desired format for analysis, allowing for the efficient manipulation of data in Pandas.



## Merging DataFrames

>Merging is an important function in Pandas that allows you to combine different sets of data. The `.merge()` function is used to bring together data from separate sources based a shared key column. This technique is particularly useful when dealing with datasets from different systems or sources that need to be analysed together.


### Basic Merge

Merging two `DataFrame`s is often based on one or more common columns. The default merge type is an **inner** join, which combines only the rows with matching values in both `DataFrames`.

In the example below, we have a `DataFrame` of customer support issues raised, the feedback score, and the outcome of the issue. We also have a `DataFrame` of data about which product each customer purchased. The `CustomerID` column is common to both tables.  We can use the `pd.merge()` function with its default settings and with `CustomerID` as the merge **key**, to produce a table with one row per support issue, but adding the data about which product the customer bought. 


In [15]:

import pandas as pd

# Customer data from the Sales Department
customer_data = pd.DataFrame({
    'CustomerID': ['C001', 'C002', 'C003', 'C004', 'C005', 'C006'],
    'Product': ['Laptop', 'Printer', 'Tablet', 'Monitor', 'Tablet', 'Laptop'],
})

# Extended Customer feedback and issues data from the Customer Service Department
service_data = pd.DataFrame({
    'CustomerID': ['C001', 'C001', 'C002', 'C003', 'C003', 'C004'],
    'Issue_ID': ['I001', 'I002', 'I003', 'I004', 'I005', 'I006'],
    'Feedback_Score': [4, 3, 3, 5, 4, 2],
    'Issue_Resolved': ['Yes', 'No', 'Yes', 'No', 'Yes', 'Yes']
})

In [16]:
customer_data

,CustomerID,Product
0,C001,Laptop
1,C002,Printer
2,C003,Tablet
3,C004,Monitor
4,C005,Tablet
5,C006,Laptop


In [17]:
service_data

,CustomerID,Issue_ID,Feedback_Score,Issue_Resolved
0,C001,I001,4,Yes
1,C001,I002,3,No
2,C002,I003,3,Yes
3,C003,I004,5,No
4,C003,I005,4,Yes
5,C004,I006,2,Yes


In [18]:
# Merging the Sales and Customer Service data on 'CustomerID'
merged_customer_data = pd.merge(service_data, customer_data, on='CustomerID')
merged_customer_data.head()

,CustomerID,Issue_ID,Feedback_Score,Issue_Resolved,Product
0,C001,I001,4,Yes,Laptop
1,C001,I002,3,No,Laptop
2,C002,I003,3,Yes,Printer
3,C003,I004,5,No,Tablet
4,C003,I005,4,Yes,Tablet



### Types of Joins

- **Inner Join:** Retrieves rows with matching values in both `DataFrames`
- **Left Join:** Includes all rows from the left `DataFrame` and matching rows from the  key column of the right `DataFrame`
- **Right Join:** Includes all rows from the right `DataFrame` and matching rows from the key column of the left `DataFrame`
- **Full (Outer) Join:** Combines all rows from both `DataFrames` where there is a match in either key column

Note that all types except for `inner` might produce missing values. Run the code cells below to see how the join types differ.

In [19]:
# Left Join
left_join_df = pd.merge(service_data, customer_data, how='left', on='CustomerID')
left_join_df

,CustomerID,Issue_ID,Feedback_Score,Issue_Resolved,Product
0,C001,I001,4,Yes,Laptop
1,C001,I002,3,No,Laptop
2,C002,I003,3,Yes,Printer
3,C003,I004,5,No,Tablet
4,C003,I005,4,Yes,Tablet
5,C004,I006,2,Yes,Monitor


The left join looks the same as the `inner` join here, because all of the customer IDs in `service_data` are also present in `customer_data`.

In [20]:

# Right Join
right_join_df = pd.merge(service_data, customer_data, how='right', on='CustomerID')
right_join_df

,CustomerID,Issue_ID,Feedback_Score,Issue_Resolved,Product
0,C001,I001,4.0,Yes,Laptop
1,C001,I002,3.0,No,Laptop
2,C002,I003,3.0,Yes,Printer
3,C003,I004,5.0,No,Tablet
4,C003,I005,4.0,Yes,Tablet
5,C004,I006,2.0,Yes,Monitor
6,C005,NaN,NaN,NaN,Tablet
7,C006,NaN,NaN,NaN,Laptop


The right join has `NaN` values in the merged table, because there are values of `CustomerID` in `customer_data` that are not present in `service_data`.

In [21]:

# Full Join
full_join_df = pd.merge(service_data, customer_data, how='outer', on='CustomerID')
full_join_df

,CustomerID,Issue_ID,Feedback_Score,Issue_Resolved,Product
0,C001,I001,4.0,Yes,Laptop
1,C001,I002,3.0,No,Laptop
2,C002,I003,3.0,Yes,Printer
3,C003,I004,5.0,No,Tablet
4,C003,I005,4.0,Yes,Tablet
5,C004,I006,2.0,Yes,Monitor
6,C005,NaN,NaN,NaN,Tablet
7,C006,NaN,NaN,NaN,Laptop


In this instance, the merged table for the full `outer` join looks the same as the `right` join table. They would look different if there were customers in the `service_data` table that weren't present in the `customer_data` table.

### Merging on Multiple Columns

Merging on multiple columns is useful when a single key isn't enough to accurately join data. For instance, in a real-world scenario, you might have data from two different departments of a company, where each department uses a combination of `Employee ID` and `Department ID` to uniquely identify records. In such cases, merging on both these columns would ensure accurate alignment of data related to specific employees in specific departments.


In [22]:
# DataFrame with employee salaries and department IDs
employee_salaries = pd.DataFrame({
    'Employee ID': [1, 2, 1, 1],
    'Department ID': [101, 101, 102, 103],  # Notice fewer unique department IDs
    'Salary': [50000, 60000, 55000, 58000]
})

# DataFrame with employee names and department IDs
employee_names = pd.DataFrame({
    'Employee ID': [1, 2, 1, 1],
    'Department ID': [101, 101, 102, 103],
    'Employee Name': ['Alice', 'Bob', 'Charlie', 'Diana']
})

# Merging on 'Employee ID' and 'Department ID'
merged_df = pd.merge(employee_salaries, employee_names, on=['Employee ID', 'Department ID'])
merged_df.head()




,Employee ID,Department ID,Salary,Employee Name
0,1,101,50000,Alice
1,2,101,60000,Bob
2,1,102,55000,Charlie
3,1,103,58000,Diana
